In [2]:
import pandas as pd
import urllib.request
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns

url = "https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-scientist/P2/fr.openfoodfacts.org.products.csv.zip"
urllib.request.urlretrieve(url, "./fr.openfoodfacts.org.products.csv.zip")

('./fr.openfoodfacts.org.products.csv.zip',
 <http.client.HTTPMessage at 0x18661bf4320>)

In [3]:
with zipfile.ZipFile("./fr.openfoodfacts.org.products.csv.zip", 'r') as zip_ref:
    with zip_ref.open('fr.openfoodfacts.org.products.csv') as file:
        data = pd.read_csv(file, sep="\t", low_memory=False)

In [4]:
varList = data.columns.tolist()
sizeData = data.shape[0]
print(varList)

['code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'product_name', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'brands', 'brands_tags', 'categories', 'categories_tags', 'categories_fr', 'origins', 'origins_tags', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_fr', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_fr', 'ingredients_text', 'allergens', 'allergens_fr', 'traces', 'traces_tags', 'traces_fr', 'serving_size', 'no_nutriments', 'additives_n', 'additives', 'additives_tags', 'additives_fr', 'ingredients_from_palm_oil_n', 'ingredients_from_palm_oil', 'ingredients_from_palm_oil_tags', 'ingredients_that_may_be_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil', 'ingredients_that_may_be_from_palm_oil_tags', 'nutrition_grade_uk', 'nutrition_grade_fr', 'pnns_groups_1', 

In [5]:
startNut = varList.index("energy_100g")
print(startNut)

63


In [6]:
size = data[data.duplicated()].shape[0]
if size == 0:
    print("No exact duplicated")
else:
    print(f"exact duplicated :{size}")
subset = ['code']
size = size_subset = data.dropna(subset=subset).duplicated(subset=subset).sum()
if size == 0:
    print(f"No possible duplicated with subset : {subset}")
else:
    print(f"exact duplicated with subset {subset} :{size}")
subset = ['product_name', 'creator', 'quantity']
size = size_subset = data.dropna(subset=subset).duplicated(subset=subset).sum()
if size == 0:
    print(f"No possible duplicated with subset : {subset}")
else:
    print(f"Possible duplicated with subset {subset} :{size}")
subset = ['product_name', 'brands', 'quantity', 'categories', 'countries']
size = size_subset = data.dropna(subset=subset).duplicated(subset=subset).sum()
if size == 0:
    print(f"No possible duplicated with subset : {subset}")
else:
    print(f"Possible duplicated with subset {subset} :{size}")


No exact duplicated
No possible duplicated with subset : ['code']
Possible duplicated with subset ['product_name', 'creator', 'quantity'] :1939
Possible duplicated with subset ['product_name', 'brands', 'quantity', 'categories', 'countries'] :138


In [8]:
missingRate = data.iloc[:, :startNut].isnull().sum()/sizeData
print(missingRate.shape[0])
print(missingRate[missingRate >= 0.5])

63
generic_name                                  0.835413
quantity                                      0.673229
packaging                                     0.753844
packaging_tags                                0.753841
categories                                    0.736854
categories_tags                               0.736919
categories_fr                                 0.736850
origins                                       0.930823
origins_tags                                  0.930938
manufacturing_places                          0.886209
manufacturing_places_tags                     0.886228
labels                                        0.854853
labels_tags                                   0.854588
labels_fr                                     0.854520
emb_codes                                     0.908639
emb_codes_tags                                0.908649
first_packaging_code_geo                      0.941382
cities                                        0.999928
cities_

In [7]:
missingRateNut = data.iloc[:, startNut:].isnull().sum()/sizeData
print(missingRateNut.shape[0])
print(missingRateNut[missingRateNut >= 0.5])

99
energy-from-fat_100g     0.997328
butyric-acid_100g        1.000000
caproic-acid_100g        1.000000
caprylic-acid_100g       0.999997
capric-acid_100g         0.999994
                           ...   
cocoa_100g               0.997045
chlorophyl_100g          1.000000
carbon-footprint_100g    0.999165
glycemic-index_100g      1.000000
water-hardness_100g      1.000000
Length: 88, dtype: float64


In [9]:
def prepareData(data, targetColumn):
    if targetColumn not in data.columns:
        raise ValueError(f"{targetColumn} is not in dataFrame.")

    dataCleaned = data.dropna(subset=[targetColumn])
    y = dataCleaned[targetColumn]
    X = dataCleaned.drop(columns=[targetColumn])

    columnsKeep = X.columns[X.isnull().mean() < 0.5]
    X = X[columnsKeep]
    
    subsetDup = ['product_name', 'brands', 'quantity', 'categories', 'countries']
    subsetDup = [col for col in subsetDup if col in X.columns]
    
    if subsetDup:
        X = X.drop_duplicates(subset=subsetDup)
        y = y.loc[X.index]
    else:
        print("No target for subsetDup.")
    
    return X, y

In [11]:
try:
    XClean, y = prepareData(data, 'nutrition_grade_fr')

    print("Nb raws after clean :", len(XClean))
    print("Nb features:", XClean.shape[1])
    columnsX = XClean.columns.tolist()
    print("\n10 first features:")
    print(columnsX[:10])
    startNut = columnsX.index("energy_100g")
    print("\nFeatures nutritional:")
    print(columnsX[startNut : ])
    print("\nDistribution of the target variable:")
    print(y.value_counts(normalize=True))
    
    print("\nDescriptive statistics for some numeric columns:")
    numericColumns = [
    'energy_100g',
    'fat_100g',
    'saturated-fat_100g',
    'sugars_100g',
    'fiber_100g',
    'proteins_100g',
    'salt_100g'  # ou 'sodium_100g'
    ]
    print(f'Features needed in Nutriscoring : {numericColumns}')
    numericColumns = [col for col in numericColumns if col in XClean.columns]
    print(XClean[numericColumns].describe())

except Exception as e:
    print(f"Error : {e}")

Nb raws after clean : 205268
Nb features: 41

10 first features:
['code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'product_name', 'brands', 'brands_tags']

Features nutritional:
['energy_100g', 'fat_100g', 'saturated-fat_100g', 'trans-fat_100g', 'cholesterol_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g', 'sodium_100g', 'vitamin-a_100g', 'vitamin-c_100g', 'calcium_100g', 'iron_100g', 'nutrition-score-fr_100g', 'nutrition-score-uk_100g']

Distribution of the target variable:
nutrition_grade_fr
d    0.283152
c    0.206593
e    0.194653
a    0.161491
b    0.154111
Name: proportion, dtype: float64

Descriptive statistics for some numeric columns:
Features needed in Nutriscoring : ['energy_100g', 'fat_100g', 'saturated-fat_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g']
        energy_100g       fat_100g  saturated-fat_100g    sugars_100g  \
count  2.050830e+05  188183.000000   